In [37]:
# AIQ 구성 종목들의 PBR 계산

from langchain.vectorstores import Chroma

def get_vectordb():

    persist_directory='db'

    vectordb = Chroma( # 기존 벡터 DB 로드
        persist_directory=persist_directory,
    )
    
    return vectordb

vectordb = get_vectordb()

### 01 잠재 가치 높은
- 가치주 효과: "내재 가치 대비 낮은 가격의 주식이, 내재 가격 대비 비싼 주식보도 수익률이 높음"을 이용하는 전략.
- 사람들이 가치주(저밸류)를 기피하고 성장주(고밸류)를 선호하기 때문. 사람들이 기피했기 때문에 가치주가 되었다고도 볼 수 있다.

In [39]:
MY_ETF = "AIQ"

from core.gpt_semantic_search import get_filter

filter_list = get_filter(etf_tkr=MY_ETF)

In [42]:
docs_list = vectordb.get(where={'$or': filter_list})

- PBR 하위(저평가) 20%

In [60]:
def get_lowest_PBR_stks(docs_list):

    PBR_KEY = "priceToBook"
    SYMBOL_KEY = "symbol"

    pbr_list = []

    for docs in docs_list['metadatas']:
        if PBR_KEY in docs.keys():
            pbr_list.append((docs[PBR_KEY], docs[SYMBOL_KEY]))

    pbr_list.sort()
    pbr_list[:round(len(pbr_list)*0.2)]

[(0.19906114, 'BIDU'),
 (0.27867314, 'STNE'),
 (0.5261802, 'BZ'),
 (0.5334392, 'VSAT'),
 (1.0539666, 'HPE'),
 (1.0781606, 'TWLO'),
 (1.3169729, 'DXC'),
 (1.3784668, 'INTC'),
 (1.5226084, 'ERIC'),
 (2.1189275, 'OKTA'),
 (2.6029758, 'VRNT')]

### 02 꾸준히 우상향 하는

- 모멘텀 좋은 주식들입니다

$
\text{K-ratio} = {\text{Gradient of Cumulative Returs} \over \text{Standard Error}}
$

In [105]:
from data.get_data import get_prices_df

import statsmodels.api as sm
import numpy as np


def get_momentums(filter_list):

    def get_k_ratio(prices_df):

        prices_df = prices_df.dropna()

        ret = prices_df.pct_change().iloc[1:]

        ret_cum = np.log(1+ret).cumsum()

        x = np.array(range(len(ret)))
        y = ret_cum.iloc[:].values

        reg = sm.OLS(y, x).fit()

        return (reg.params/reg.bse) # K-ratio


    tickers = [fltr['symbol'] for fltr in filter_list]

    prices_df = get_prices_df(
        tickers=tickers,
        start_date="2022-08-29"
    )

    kratio_list = []

    for t in tickers:

        kratio = get_k_ratio(prices_df=prices_df[t.lower()])

        kratio_list.append((round(kratio[0], 2), t))

    kratio_list.sort(reverse=True)

    return kratio_list[:round(len(kratio_list)*0.2)]

  0%|          | 0/59 [00:00<?, ?it/s]

[(103.16, 'META'),
 (88.03, 'NVDA'),
 (75.48, 'TTD'),
 (75.28, 'AAPL'),
 (67.96, 'CDNS'),
 (64.71, 'ORCL'),
 (64.32, 'SNPS'),
 (60.47, 'CRM'),
 (58.74, 'AMZN'),
 (57.23, 'UBER'),
 (51.63, 'GOOGL'),
 (49.79, 'TRI')]